## file for testing different api / individual components of the whole server setup

In [1]:
## cleaning up imports, given below are the imports used by the main file
# idling
# eyescontroller
# trackhead
# detect
# facetracker
# listen
# speak
# classifyquery
# convflow
# returntodefaultpos
# stoplistening

In [1]:
# checking if their individual imports work
from idle import idling

/usr/local/lib/python2.7/site-packages/paramiko/transport.py:33: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography.hazmat.backends import default_backend


In [2]:
from interactiveControls import EyesController
from interactiveControls import track_head
from interactiveControls import return_to_default_pos

In [3]:
from humanInteraction import detect
from humanInteraction import listen
from humanInteraction import speak
from humanInteraction import faceTracker
from humanInteraction import classifyQuery
from humanInteraction import stopListening

In [4]:
from chatflow import convFlow

In [5]:
idle = idling()
idle.start_idle_behavior()

In [6]:
# testing casual query api
idle.stop()
from humanInteraction import postCasualQuery
from interactiveControls import EyesController
from time import time

start = time()
postCasualQuery('i need some help finding the toilet can you show me the way', [], EyesController())
idle.start_idle_behavior()
end = time()
print('Time taken: ', end - start)

DG: onlyTextAndImgPage.html copyfile successful
DG: HeadText text match found
DG: BodyText text match found
Sent to Pepper
Sent to Pepper
[]
I'm sorry but I am not able to leave my post during this event and also Pepper robots are designed with safety in mind,
 so it's highly unlikely that anyone would touch us or encourage people.
 However if someone needs any assistance related to physical queries like showing them a map of the campus,
 please feel free to ask me for help!
quit
('Time taken: ', 49.600841999053955)


## flow for processQuery

`processQuery(query, responsesPipeline, eyes, state)`<br>
**| <br>
|--** `classifyQuery(query)`<br>
**|<br> 
|--** Case: `query` starts with "%"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for confusion and repeat<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `showWhichPage("confused")`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `eyes.setEyes("confused")`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `speak(tieredConfusionPrompts[state["confusion"]])`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for `%error%` topic<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**| <br>
|--** Case: `topic[:3]` in ["Acco", "Acti", "Camp"] and not confident<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** `verifyTopic(topic, eyes)`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Verify `topic` with user and update `confident`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `confident` or `topic[:3]` in ["Cour", "Spec", "Gene"]<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topic[:3]` is "Cour" and `previousTopic` is "Cour"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Set `topic` to "Spec"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `topicSpecificOutput(topic, query, responsesPipeline, eyes)`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topicSpecificOutput` errored<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for repetition and confusion<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for repetition and confusion<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `confident` or `topic[:3]` not met<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for confusion<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `showWhichPage("confused")`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `eyes.setEyes("confused")`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `speak(confusionRepeat[1])`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for repetition<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` with `topic`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**| <br>
|--** `topicSpecificOutput(topic, query, responsesPipeline, eyes)`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topic` in ["Acco", "Acti", "Camp"] and not confident<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `verifyTopic(topic, eyes)`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Verify topic with user and update `confident`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topic` is confident or "Cour", "Spec", "Gene"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topic` is "Cour" and `previousTopic` is "Cour"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Set `topic` to "Spec"<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** `topicSpecificOutput(topic, query, responsesPipeline, eyes)`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topicSpecificOutput` errored<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for repetition and confusion<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` with `topic` and confidence<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Case: `topic` is not confident<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` for confusion<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;|--** Show confusion message and update `currentState` for repetition<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**|&nbsp;&nbsp;&nbsp;&nbsp;|--** Update `currentState` with `topic`<br>
**|&nbsp;&nbsp;&nbsp;&nbsp;.** <br>
**.**

In [7]:
idle.stop()
from humanInteraction import classifyQuery

# testing classification
query1 = 'what are some masters courses you offer in data science'  # query for class courses
query2 = 'where can i stay at deakin'  # query for class accommodation
query3 = 'are there any sports clubs'  # query for class activities
query4 = 'how are you doing'           # query for class casual
query5 = "%"                           # error query

print 'query: ', query1, ':', classifyQuery(query1)
print 'query: ', query2, '\t\t\t\t:', classifyQuery(query2)
print 'query: ', query3, '\t\t\t\t:', classifyQuery(query3)
print 'query: ', query4, '\t\t\t\t\t:', classifyQuery(query4)
print 'query: ', query5, '\t\t\t\t\t\t\t:', classifyQuery(query5)
idle.start_idle_behavior()

# the first element should be a four letter code for the type
# belonging to Cacc, Cour, Club, Cgen

query:  what are some masters courses you offer in data science : ('Cour', True)
query:  where can i stay at deakin 				: ('Cacc', True)
query:  are there any sports clubs 				: ('Club', True)
query:  how are you doing 					: ('Cgen', True)
query:  % 							: ('Cacc', False)


In [8]:
idle.stop()
# testing topicSpecificOutput
from interactiveControls import EyesController
eyes = EyesController()

from humanInteraction import topicSpecificOutput

print 'topicSpecificOutput', 'query1', topicSpecificOutput(classifyQuery(query1)[0], query1, [], eyes)
print 'topicSpecificOutput', 'query2', topicSpecificOutput(classifyQuery(query2)[0], query2, [], eyes)
print 'topicSpecificOutput', 'query3', topicSpecificOutput(classifyQuery(query3)[0], query3, [], eyes)
print 'topicSpecificOutput', 'query4', topicSpecificOutput(classifyQuery(query4)[0], query4, [], eyes)
idle.start_idle_behavior()

topicSpecificOutput query1 DG: basicListViewPage.html copyfile successful
 <li> Master of Data Science (Professional) </li>
 <li> Bachelor of Data Science </li>
 <li> Master of Data Science </li>
 <li> Graduate Diploma of Data Science </li>
 <li> Bachelor of Data Science (Honours) </li>

DG: ListText text match found
Sent to Pepper
Sent to Pepper
False
topicSpecificOutput query2 DG: bottomBannerWithBodyQRPage.html copyfile successful
DG: HeadText text match found
DG: BodyText text match found
Sent to Pepper
Sent to Pepper
Sent to Pepper
False
topicSpecificOutput query3 DG: bottomBannerWithBodyQRPage.html copyfile successful
DG: HeadText text match found
DG: BodyText text match found
Sent to Pepper
Sent to Pepper
Sent to Pepper
False
topicSpecificOutput query4 DG: onlyTextAndImgPage.html copyfile successful
DG: HeadText text match found
DG: BodyText text match found
Sent to Pepper
Sent to Pepper
[]
I'm great!
quit
False


In [ ]:
def print_dict_with_indent(dct, indent=0, state_type='NEW'):
    start_print = '{}_state ='.format(state_type)
    print start_print, ' {'
    for key, value in dct.items():
        if isinstance(value, dict):
            print("  " * indent + "'{}': {{".format(key))
            print_dict_with_indent(value, indent + 1)
            print("  " * indent + "},")
        else:
            print("  " * indent + "'{}': {},".format(key, value))
    print('}')


# tracking possible state values 
states = {
    "topic": [
        "fresh",    # default, 
        "%error%",  # line 73 - humanInteraction.py
        "Cacc",     # line 99 - humanInteraction.py
        "Cour",     # line 99 - humanInteraction.py
        "Club",     # line 99 - humanInteraction.py
        "Camp",     # line 99 - humanInteraction.py
        "Cgen",     # line 99 - humanInteraction.py
        "Cspe",     # line 86 - humanInteraction.py
    ], 
    "confusion": [
        0,          # default, 
        1,2,3,4     # line 68 90 94 - humanInteraction.py
    ], 
    "repeat": [
        True,       # default, line 73 89 98 - humanInteraction.py, 
        False,      # line 66 - humanInteraction.py
    ], 
    "confident": [
        False,      # default
    ],
}

# testing process query with different state combinations
from itertools           import product
from interactiveControls import EyesController
from humanInteraction    import processQuery
state_combinations = product(*states.values())
# same initial state is passed everywhere
for state_values in state_combinations:
    state_dict = dict(zip(states.keys(), state_values))
    print_dict_with_indent(state_dict, 2, "INITIAL")
    
    returned_state = processQuery(query1, [], eyes, state_dict)
    print 'QUERY NUMBER 1'
    print_dict_with_indent(returned_state, 2, )
    
    returned_state = processQuery(query2, [], eyes, state_dict)
    print 'QUERY NUMBER 2'
    print_dict_with_indent(returned_state, 2, )
    
    returned_state = processQuery(query3, [], eyes, state_dict)
    print 'QUERY NUMBER 3'
    print_dict_with_indent(returned_state, 2, )
    
    returned_state = processQuery(query4, [], eyes, state_dict)
    print 'QUERY NUMBER 4'
    print_dict_with_indent(returned_state, 2, )
    
    returned_state = processQuery(query5, [], eyes, state_dict)
    print 'QUERY NUMBER 5'
    print_dict_with_indent(returned_state, 2,)
    

INITIAL_state =  {
    'topic': fresh,
    'confusion': 0,
    'repeat': True,
    'confident': False,
}
('QUERY CLASS: ', 'Cour', True)
DG: basicListViewPage.html copyfile successful
 <li> Master of Data Science (Professional) </li>
 <li> Bachelor of Data Science </li>
 <li> Master of Data Science </li>
 <li> Graduate Diploma of Data Science </li>
 <li> Bachelor of Data Science (Honours) </li>

DG: ListText text match found
Sent to Pepper
Sent to Pepper
QUERY NUMBER 1
NEW_state =  {
    'topic': Cour,
    'confusion': 0,
    'repeat': False,
    'confident': False,
}
('QUERY CLASS: ', 'Cacc', True)
DG: bottomBannerWithBodyQRPage.html copyfile successful
DG: HeadText text match found
DG: BodyText text match found
Sent to Pepper
Sent to Pepper
Sent to Pepper
QUERY NUMBER 2
NEW_state =  {
    'topic': Cacc,
    'confusion': 0,
    'repeat': False,
    'confident': False,
}
('QUERY CLASS: ', 'Club', True)
DG: bottomBannerWithBodyQRPage.html copyfile successful
DG: HeadText text match foun